### Regresión Logística - Pair programming ###

## 1. EDA ##

In [ ]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd


# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Gestión de warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

Para los ejercicios de pair programming de Regresión logística tendremos que buscar un dataset (al igual que hicismos en regresión lineal) que usaremos a lo largo de los siguientes ejercicios.

Se ruega a la hora de realizar la entrega que incluyais el conjunto de datos que hayais decidido emplear para estos ejercicios.

Objetivos

Buscar un conjunto de datos a analizar

Se recomienda que el conjunto de datos a analizar tenga variables numéricas y categóricas, primando que haya más de una variable de tipo numérico.

Explicar los datos y las variables disponibles en el conjunto de datos seleccionado

Realizar un EDA sencillo poniendo en práctica los conocimientos adquiridos hasta el momento.

Interpretación de los resultados.

In [ ]:
df = pd.read_csv("data/card_transdata.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

Vemos que no tenemos ni nulos ni duplicados.

In [ ]:
correlacion = df.corr()
correlacion

In [ ]:
plt.figure(figsize=(15,8))
mask = np.triu(np.ones_like(correlacion, dtype=bool))
sns.heatmap(correlacion, cmap="viridis", annot = True, mask=mask, vmin = -1, vmax=1);

Podemos observar que todas nuestras variables son independientes

In [ ]:
df.describe().T

Vemos que en las variables numéricas tendremos outliers, que los estudiaremos más adelante.

In [ ]:
df["fraud"].value_counts()

In [ ]:
df["online_order"].value_counts()

In [ ]:
df["used_pin_number"].value_counts()

In [ ]:
df["used_chip"].value_counts()

In [ ]:
df["repeat_retailer"].value_counts()

Cambiamos las columnas a categóricas porque no consideramos que sean numéricas.

In [ ]:
columnas_cat = ["used_chip", "repeat_retailer", "used_pin_number", "online_order", "used_pin_number", "fraud"]
for columna in columnas_cat:
    df[columna] = df[columna].astype("category")

In [ ]:
df.info()

In [ ]:
df["distance_from_home"].value_counts()

In [ ]:
#Gráfica para ver la variable respuesta
sns.countplot(data= df, x = "fraud");

In [ ]:
df.head()

In [ ]:
df.groupby('fraud')['distance_from_home'].mean().reset_index()

In [ ]:
df.groupby('fraud')['distance_from_home'].median().reset_index()

In [ ]:
df.groupby('fraud')['distance_from_last_transaction'].mean().reset_index()

In [ ]:
df.groupby('fraud')['ratio_to_median_purchase_price'].median().reset_index()

Hemos observado que hay ciertos patrones que se repiten, como que las compras fraudulentas se hacen a más distancia, en compras online y con una diferencia de importe.

In [ ]:

fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (30, 20))

axes = axes.flat
columnas_cat = df.select_dtypes(include = "category").columns

for i, colum in enumerate(columnas_cat): 
    sns.countplot(
        data = df,
        x = colum,
        hue = "fraud",
        alpha = 0.2, 
        ax = axes[i])
    
    
    axes[i].set_title(colum, fontsize = 15, fontweight = "bold")
    axes[i].tick_params(labelsize = 20)
    axes[i].set_xlabel("")
    axes[i].legend(loc= "upper right")

fig.tight_layout();

De estos gráficos deducimos que los fraudes suelen ser en pagos recurrentes en compra online y sin usar el número pin.

In [ ]:

fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (30, 10))

lista_colores = ["blue", "green", "orange"]

axes = axes.flat

columnas_numeric = df.select_dtypes(include = np.number).columns

for i, colum in enumerate(columnas_numeric): 
    sns.histplot(
        data = df,
        x = colum,
        kde = True, 
        color = lista_colores[i], 
        line_kws = {"linewidth": 2}, 
        alpha = 0.2, 
        ax = axes[i])
    
    
    axes[i].set_title(colum, fontsize = 15, fontweight = "bold")
    axes[i].tick_params(labelsize = 20)
    axes[i].set_xlabel("")
    
fig.tight_layout();

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (30, 10))

lista_colores = ["blue", "green", "orange"]

axes = axes.flat

columnas_numeric = df.select_dtypes(include = np.number).columns

for i, colum in enumerate(columnas_numeric): 
    sns.histplot(
        data = df,
        x = colum,
        hue = "fraud",
        color = lista_colores[i], 
        line_kws = {"linewidth": 2}, 
        alpha = 0.2, 
        ax = axes[i])
    
    
    axes[i].set_title(colum, fontsize = 15, fontweight = "bold")
    axes[i].tick_params(labelsize = 20)
    axes[i].set_xlabel("")
    
fig.tight_layout();

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30,10))

categorias = df.select_dtypes(include = np.number).columns

axes = axes.flat

for indice, cate in enumerate(categorias):

    sns.boxplot(x = df[cate], data = df, ax=axes[indice], color = "cyan"); # para trazar una línea de densidad

    axes[indice].set_title(cate)
    axes[indice].set_xlabel("")

fig.tight_layout();

In [ ]:
df.head(1)

In [ ]:
def detectar_outliers(lista_columnas, dataframe): 
    
    dicc_indices = {} # creamos un diccionario donde almacenaremos índices de los outliers
    
    # iteramos por la lista de las columnas numéricas de nuestro dataframe
    for col in lista_columnas:
        
        #calculamos los cuartiles Q1 y Q3
        Q1 = np.nanpercentile(df[col], 25)
        Q3 = np.nanpercentile(df[col], 75)
        
        # calculamos el rango intercuartil
        IQR = Q3 - Q1
        
        # calculamos los límites
        outlier_step = 1.5 * IQR
        
        # filtramos nuestro dataframe para indentificar los outliers
        outliers_data = dataframe[(dataframe[col] < Q1 - outlier_step) | (dataframe[col] > Q3 + outlier_step)]
        
        
        if outliers_data.shape[0] > 0: # chequeamos si nuestro dataframe tiene alguna fila. 
        
            dicc_indices[col] = (list(outliers_data.index)) # si tiene fila es que hay outliers y por lo tanto lo añadimos a nuestro diccionario
        

    
    return dicc_indices 

In [ ]:
ind = detectar_outliers(['distance_from_home', 'distance_from_last_transaction', 'ratio_to_median_purchase_price'], df)
ind

In [ ]:
valores = ind.values()
valores = {indice for sublista in valores for indice in sublista}
valores

In [ ]:
df_outliers = df[df["index"].isin(valores)]

In [ ]:
df[df.index.isin(valores)]

In [ ]:
#df.reset_index(inplace= True)

In [ ]:
df_outliers['fraud'].value_counts()

In [ ]:
df.to_pickle("data/card_limpio.pkl")